# Get a LSST Simulated Cadence for Arbitrary Sky Position
___


___

In [ ]:
"""

"""
%matplotlib inline

from __future__ import print_function

try:
    from importlib import reload
except:
    pass


from matplotlib import pyplot as plt
import matplotlib.colors as mpl_colors 

import os
import warnings

import numpy as np
import pandas as pd
import astropy as ap
import sfdmap

import sqlite3
from sqlalchemy import create_engine

from astropy import units as u
from astropy.coordinates import SkyCoord

import lsst_tools.utils as utils 
import lsst_tools.utils.colours as colours 

import pyCoCo as pccsims

Connect to .db file that contains the opsim output, read into a dataframe, `opsimdf`

In [ ]:
# %%timeit ## 1 loop, best of 3: 1min 31s per loop 
# opsimdbpath = os.environ.get('OPSIMDBPATH')
# print(opsimdbpath)
opsimdbpath = "/Users/berto/data/LSST/OpSimOutputDBs/astro_lsst_01_1068_sqlite.db"

conn = create_engine('sqlite:///'+opsimdbpath, echo = False)
opsimdf = pd.read_sql_table('Summary', con=conn)

In [ ]:
opsimdf

In [ ]:
#%%timeit ## The slowest run took 39.04 times longer than the fastest. This could mean that an intermediate result is being cached.
#         ## 1000 loops, best of 3: 246 µs per loop
n = 1
pos_df = utils.generate_coordinates(n)

Add extinction to the dataframe

In [ ]:
#%%timeit ## 1 loop, best of 3: 88.6 ms per loop
df = utils.find_MW_extinction(pos_df) 

Check the positions are sensible

In [ ]:
#%%timeit ## 1 loop, best of 3: 235 ms per loop
utils.plot_position_points(df)

Connecting to `.db` takes ages (~1min), and is a pain if you mess up, so create a 'working' instance to fiddle with

In [ ]:
working_df = opsimdf

## Identifying Fields
___
We want to find out which field(s) the position is in. Create table that tells us the field centres, and find the edges.

In [ ]:
# working_df = utils.get_field_corners(working_df.drop_duplicates("fieldID"))[['fieldRA', 'fieldDec']]
working_df = utils.get_field_corners(pd.DataFrame(working_df.drop_duplicates("fieldID"))[['fieldID', 'fieldRA', 'fieldDec']])

In [ ]:
working_df

In [ ]:
field_df = working_df[(working_df["RA_upper"] >= df["theta"][0] + np.pi) &
                (working_df["RA_lower"] <= df["theta"][0] + np.pi) &
                (working_df["Dec_lower"] >= df["phi"][0])  &
                (working_df["Dec_upper"] <= df["phi"][0])]

In [ ]:
reload(utils)
utils.plot_field(field_df)

In [ ]:
field_df

In [ ]:
mjd_to_sim = opsimdf.loc[opsimdf["fieldID"] == 2846,["expMJD"]].values.T[0]
filters_to_sim = opsimdf.loc[opsimdf["fieldID"] == 2846,["filter"]].values.T[0]
filters_to_sim = np.array([b('LSST_'+x) for x in filters_to_sim])

In [ ]:
# mjdmax = np.random.choice(mjd_to_sim)
mjdmax = np.random.choice(mjd_to_sim)

In [ ]:
mjdmax

In [ ]:
filters_to_sim

In [ ]:
import pyCoCo as pccsims

In [ ]:
import sys
if sys.version_info < (3,):
    def b(x):
        return x
else:
    import codecs
    def b(x):
        return codecs.latin_1_encode(x)[0]

In [ ]:
filter_path = "/Users/berto/Code/CoCo/data/filters"
coco_root_path = "/Users/berto/Code/CoCo"


coco = pccsims.pyCoCo(b(filter_path), b(coco_root_path))

In [ ]:
# mjd_to_sim = np.array([90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 105.0, 110.0, 115.0, 120.0])
# filters_to_sim = np.array([b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r'])
# mjd_to_sim = 
# filters_to_sim = 

In [ ]:
z_obs = 0.007


In [ ]:
out = coco.simulate(b"SN2009jf", 
                    z_obs, -0.1, 0.5, 0.5, 3.1, 
                    mjdmax, mjd_to_sim, 
                    filters_to_sim)

In [ ]:
out

In [ ]:
def simulate_out_to_ap_table(mjd_to_sim, flux, dflux, filters_to_sim, 
                             names = ('MJD', 'flux', 'flux_err', 'filter')):
    return Table([mjd_to_sim, flux, dflux, filters_to_sim.astype(str)], names = names)

In [ ]:
import pycoco as pcc
from astropy.table import Table
reload(pcc)

In [ ]:
p = pcc.PhotometryClass()
ptable = simulate_out_to_ap_table(mjd_to_sim, out[0], out[1], filters_to_sim)

In [ ]:
p.phot = ptable

In [ ]:
p.unpack()

In [ ]:
p.plot()

In [ ]:
# mjdmax = testmjd[w[0]][0]
z_obs = 0.007
# mjd_to_sim = np.array(sn.phot.phot["MJD"])
# filters_to_sim = np.array([b(i.astype(str)) for i in sn.phot.phot["filter"]])

mjdmax = 100.0
mjd_to_sim = np.array([90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 105.0, 110.0, 115.0, 120.0])
filters_to_sim = np.array([b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r', b'SDSS_r'])
print(mjdmax)
print(mjd_to_sim)
print(filters_to_sim)

In [ ]:
out = coco.simulate(b"SN2009jf", 
                    z_obs, -0.0, 0.5, 0.5, 3.1, 
                    mjdmax, mjd_to_sim, 
                    filters_to_sim)

In [ ]:
out